### 18.2. 문자열 생성

셰익스피어 데이터셋 주소

https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt 

In [1]:
using Flux
using Flux.Losses: logitcrossentropy
import Zygote, Optimisers
using MLUtils: chunk, batchseq
using StatsBase: wsample
using Formatting: printfmtln
using Random: MersenneTwister

In [2]:
fpath = "tinyshakespeare.txt"
furl = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
isfile(fpath) || download(furl, fpath)

true

데이터셋 생성

In [3]:
function get_data(fpath; batch_size = 32, seq_len = 100)    
    text = collect(String(read(fpath)))
    alphabet = unique(text)
    '_' in alphabet || push!(alphabet, '_');
    
    text = map(ch -> Flux.onehot(ch, alphabet), text)
    pad = Flux.onehot('_', alphabet);

    Xs = batchseq(chunk(text, batch_size), pad)
    Xs = map(x -> hcat(x...), Xs)
    Xs = chunk(Xs; size = seq_len)#[1:end-1];

    Ys = batchseq(chunk(text[2:end], batch_size), pad)
    Ys = map(y -> hcat(y...), Ys)
    Ys = chunk(Ys; size = seq_len)#[1:end-1]

    zip(Xs, Ys), alphabet
end

get_data (generic function with 1 method)

In [4]:
fpath = "tinyshakespeare.txt"
loader, alphabet = get_data(fpath, batch_size = 32, seq_len = 100);

학습 함수 (16장 학습 함수와 동일)

In [5]:
function train(loader, model, loss_fn, optimizer)
    num_batches = length(loader)
    Flux.testmode!(model, false)
    for (batch, (X, y)) in enumerate(loader)
        X, y = Flux.gpu(X), Flux.gpu(y)
        grad = Zygote.gradient(m -> loss_fn(m, X, y), model)[1]
        optimizer, model = Optimisers.update(optimizer, model, grad)
        if batch % 100 == 0
            loss = loss_fn(model, X, y)
            printfmtln("[Train] loss: {:.7f} [{:>3d}/{:>3d}]", 
                loss, batch, num_batches)
        end
    end
    model, optimizer
end

train (generic function with 1 method)

모델 및 손실 함수

In [6]:
init(rng) = Flux.glorot_uniform(rng)

function build_model(N; rng)
    Chain(
        LSTM(N, 512; init=init(rng)),
        LSTM(512, 512; init=init(rng)),
        Dense(512, N; init=init(rng))
    )
end;

rng = MersenneTwister(1)
model = build_model(length(alphabet); rng=rng) |> gpu;
loss_fn(m, xs, ys) = sum(logitcrossentropy.([m(x) for x in xs], ys));
optimizer = Optimisers.setup(Optimisers.Adam(), model);

In [7]:
for epoch in 1:20
    Flux.reset!(model)
    println("Epoch $epoch")
    println("-------------------------------")
    global model, optimizer = train(loader, model, loss_fn, optimizer)
end

Epoch 1
-------------------------------


[Train] loss: 3

17.8969727 [100/349]


[Train] loss: 245.4123840 [200/349]


[Train] loss: 227.3763275 [300/349]


...

[Train] loss: 117.0097656 [300/349]


Epoch 20
-------------------------------


[Train] loss: 121.0933533 [100/349]


[Train] loss: 119.8905106 [200/349]


[Train] loss: 116.1155472 [300/349]


In [8]:
function generate(model, alphabet, init, len; rng)    
    model = model |> cpu
    Flux.reset!(model)
    generated = [init]
    for _ in 1:len
        w = softmax(model(Flux.onehot(generated[end], alphabet)))
        push!(generated, wsample(rng, alphabet, w))
    end
    text = String(generated)
    for r in split(text, '\n')
        println(r)
    end
    text
end

generate (generic function with 1 method)

In [14]:
generate(model, alphabet, 'O', 500; rng=MersenneTwister(1));

O, Yet glove against the king:
I was a father doth paph direth.

ROMEO:
Stupp'st; My brother, till you had but fury to that.

HERMIONE:
Why, thy are,
Doth hein and leart in strifferance.

SEBASTIAN:
An I all so: but hath been so disconcedded;
'Tis a thrifty is scen-willingly go.
Came heart ucts not for you, But I by no.
Ah, so! no; I, how ly Isable, nor then;
So assured, mirrily, or Claudio, lives thustraft.

KING LEWIS XI:
What is this?

First Citizen:
We are, go you at your children, sir; yet I


In [12]:
using BSON: @save
@save "tinyshakespeare.bson" model = Flux.cpu(model), alphabet